In [63]:
import pandas as pd
import numpy as np
import panel as pn
import datetime as dt
from panel.interact import interact

pn.extension('tabulator')
import hvplot.pandas

# Interactive dash board with panel
In this project, I want to explore the panel package in python and make a interactive dashboard. By doing that, we need to create widgets we want to display on panel and create pipelines to reflect data change on panels. (Widgets - Interactive Pipeline - Dataframe) The source data is obtained from the world data bank, which contains country population from 1960 to 2021.

In [64]:
#data source: https://data.worldbank.org/indicator/SP.POP.TOTL

# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df =  pd.read_csv('pop.csv')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

# Data preprocessing
Firstly, I will obtain a list of countries for later use and we sort the list in alphabetical order.

In [65]:
countries = df['Country Name'].values.tolist()
countries.sort()
df.head()

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Afghanistan,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,10174840.0,10399936.0,10637064.0,...,31161378,32269592,33370804,34413603,35383028,36296111,37171922,38041757,38928341,39835428
1,Albania,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,...,2900401,2895092,2889104,2880703,2876101,2873457,2866376,2854191,2837849,2811666
2,Algeria,11057864.0,11336336.0,11619828.0,11912800.0,12221675.0,12550880.0,12902626.0,13275020.0,13663581.0,...,37383899,38140135,38923688,39728020,40551398,41389174,42228415,43053054,43851043,44616626
3,American Samoa,20127.0,20605.0,21246.0,22029.0,22850.0,23675.0,24473.0,25235.0,25980.0,...,55669,55717,55791,55806,55739,55617,55461,55312,55197,55103
4,Andorra,13410.0,14378.0,15379.0,16407.0,17466.0,18542.0,19646.0,20760.0,21886.0,...,82427,80770,79213,77993,77295,76997,77008,77146,77265,77354


 And we can see the dataframe has 'year' as column names, but we need all year values as a column. We can improve that with a melt() function. We also set new column name as 'Year' and 'Population'.

In [66]:
df = df.melt('Country Name', var_name='Year', value_name='Population')
df['Year'] = pd.to_numeric(df['Year'])
df['Population'] = pd.to_numeric(df['Population'])
df.head()

,Country Name,Year,Population
0,Afghanistan,1960,8996967.0
1,Albania,1960,1608800.0
2,Algeria,1960,11057864.0
3,American Samoa,1960,20127.0
4,Andorra,1960,13410.0


We will fill any NA value with 0 and make our dataframe pipeline interactive.

In [67]:
# Fill NAs with 0s
df = df.fillna("0")

# Make DataFrame Pipeline Interactive
idf = df.interactive()
type(idf)

hvplot.interactive.Interactive

# Create widgets
The first widget I create is a IntSlider widget which allows selecting an integer value within a set bounds using a slider.

In [68]:
year_slider = pn.widgets.IntSlider(name='Year', start=1960, end=2021, step=1, value=2000)

year_slider

IntSlider(end=2021, name='Year', start=1960, value=2000)

The second widget I have is a Select widget which allows selecting a value from a list of options by selecting it from a dropdown menu. 

In [69]:
country_select = pn.widgets.Select(name='Country', options= countries)

country_select

Select(name='Country', options=['Afghanistan', ...], value='Afghanistan')

# Create pipelines
The first pipeline simply connects the select widget to the column 'Country Name'. In that way, it returns the population of selected country in all available years. 

In [70]:
ipipeline = (
    idf[
        (idf['Country Name'] == country_select)
    ]
    .groupby(['Country Name', 'Year'])['Population'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')  
    .reset_index(drop=True)
)

ipipeline.head()

We then turn the pipeline to a table for better visualization. We can define number of entres we want to disply each page, so we won't have a huge list on our panel.

In [71]:
itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, theme=theme)
itable

Same way, we turn the pipeline to a plot which has population as y-axis and year as x-axis, for country selected by the widget.

In [72]:
iplot = ipipeline.hvplot(x = 'Year',  y='Population',line_width=2, title="Population (1960 - 2021)")
iplot

Second pipeline I have is connected to the year slider. It returns 10 countries with highest population for the year selected by year slider.

In [73]:
top10_pipeline = (
    idf[
        (idf['Year'] == (year_slider))
    ]
    .groupby(['Country Name', 'Year'])['Population'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Population',ascending=False)  
    .reset_index(drop=True)
)

top10_pipeline = top10_pipeline.head(10)
top10_pipeline

We then turn the pipeline into a table and a bar chart.

In [74]:
top10_itable = top10_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10)
top10_itable

In [75]:
top10_bar_plot = top10_pipeline.hvplot(kind='bar', x='Country Name',  y='Population', title='Top 10 countries with highest population ')
top10_bar_plot

# Layout using Template
Finally, we create our template which allows us to decide how and where we want to display the widgets, tables and plots. template.show() will open a new page in notebook which has the interactive dash board. Alternatively, we can run template.servable(), and run "serve dashboard.ipynb in terminal".

In [76]:
template = pn.template.FastListTemplate(
    title='World Population dashboard', 
    main=[pn.Row(pn.Column(country_select, iplot.panel(width=700,margin=(0,25))), itable.panel(width=700,margin=(0,25))),
          pn.Row(pn.Column(top10_bar_plot.panel(width=1100,margin=(0,25)),year_slider))
         ],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

template.show()
#template.servable()
#To serve the notebook run panel serve dashboard.ipynb.

Launching server at http://localhost:59970
